In [1]:
import torch
hw = torch.zeros((3, 2, 300, 400))
c2v = torch.ones((3, 8, 300, 400)) * 4
v2c = torch.zeros((3, 8, 300, 400)) * (-3)
diff = (c2v - v2c) / 2

In [20]:
hw[:, 0, ...] = (- diff[:, 0, ...] + diff[:, 2, ...] + diff[:, 4, ...] - diff[:, 6, ...]) /2
hw[:, 1, ...] = (- diff[:, 1, ...] - diff[:, 3, ...] + diff[:, 5, ...] - diff[:, 7, ...]) /2


In [3]:
t = torch.randn((4, 3))

torch.expit(t)

AttributeError: module 'torch' has no attribute 'expit'

In [1]:
import os.path as osp
from sys import argv

import mmcv
import wandb
from mmcv import Config
from mmdet.apis import set_random_seed, train_detector

# Let's take a look at the dataset image
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

wandb.login()

cfg = Config.fromfile(f"/home/aiarhipov/centernet/exps/20_cycle_l1/config.py")


set_random_seed(0, deterministic=False)

# Build dataset
datasets = [build_dataset(cfg.data.train), build_dataset(cfg.data.val_loss)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: archiealexarkhipov (centernet). Use `wandb login --relogin` to force relogin


loading annotations into memory...
Done (t=12.65s)
creating index...
index created!
loading annotations into memory...
Done (t=3.72s)
creating index...
index created!


2023-05-07 20:42:07,275 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.


loading annotations into memory...
Done (t=4.67s)
creating index...


2023-05-07 20:42:12,440 - mmdet - INFO - Start running, host: aiarhipov@lorien.atp-fivt.org, work_dir: /home/aiarhipov/centernet/exps/20_cycle_l1_avg_factor_8_2heatmaps_short
2023-05-07 20:42:12,442 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) TensorboardLoggerHook              
(VERY_LOW    ) MMDetWandbHook                     
 ------------------

...

2023-05-07 20:42:17,584 - mmdet - WARNING - No meta information found in the runner. 
wandb:   32 of 32 files downloaded.  
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
2023-05-07 20:51:13,915 - mmdet - INFO - Epoch [1][500/1372]	lr: 6.244e-04, eta: 2 days, 11:38:09, time: 1.046, data_time: 0.016, memory: 2448, loss_center_heatmap: 8.7718, loss_offset: 0.1174, loss_c2v: 3.4393, loss_v2c: 0.0227, loss: 12.3512, grad_norm: 72.8073
2023-05-07 21:00:13,240 - mmdet - INFO - Epoch [1][1000/1372]	lr: 1.249e-03, eta: 2 days, 12:25:36, time: 1.079, data_time: 0.010, memory: 2448, loss_center_heatmap: 1.9584, loss_offset: 0.0864, loss_c2v: 1.9862, loss_v2c: 0.0162, loss: 4.0472, grad_norm: 17.3755
2023-05-07 21:07:04,958 - mmdet - INFO - Saving checkpoint at 1 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 3611/3611, 10.5 task/s, elapsed: 344s, ETA:     0s

2023-05-07 21:18:05,805 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=45.13s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


KeyboardInterrupt: 

In [2]:
%debug

> /home/aiarhipov/mmdetection/mmdet/models/dense_heads/cycle_centernet_head.py(373)get_targets()
    371                 offset_target[batch_id, 0, cty_int, ctx_int] = ctx - ctx_int
    372                 offset_target[batch_id, 0, tl_x_int, tl_y_int] = tl_x - tl_x_int
--> 373                 offset_target[batch_id, 0, tr_x_int, tr_y_int] = tr_x - tr_x_int
    374                 offset_target[batch_id, 0, br_x_int, br_y_int] = br_x - br_x_int
    375                 offset_target[batch_id, 0, bl_x_int, bl_y_int] = bl_x - bl_x_int

torch.Size([8, 2, 128, 128])
(1, 0, 128, 4)
